# Processing NOAA Weather Data of JFK Airport (New York)

This notebook relates to the NOAA Weather Dataset - JFK Airport (New York). The dataset contains 114,546 hourly observations of 12 local climatological variables (such as temperature, wind speed and precipitation) collected at JFK airport. This dataset is freely available from the IBM Developer [Data Asset Exchange](https://developer.ibm.com/exchanges/data/all/jfk-weather-data/).

In this notebook, we process the raw dataset by:
* selecting the columns we wish to keep for later downstream tasks
* converting and cleaning data where required
* filling missing values
* extracting categorical weather features

#### Import required modules

Import and configure the required modules.

In [ ]:
# !pip install pandas > /dev/null 2>&1

In [ ]:
# Define required imports
import pandas as pd
import numpy as np
import re
# These set pandas max column and row display in the notebook
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

### Read the Raw Data

We start by reading in the raw dataset, displaying the first few rows of the dataframe, and taking a look at the columns and column types present.

In [ ]:
raw_data = pd.read_csv('data/noaa-weather-data-jfk-airport/jfk_weather.csv', parse_dates=['DATE'])
raw_data.head()

### Clean the Data

As you can see above, there are a lot of fields which are non-numerical - usually these will be fields that contain text or categorical data, e.g. `HOURLYPRSENTWEATHERTYPE`.

There are also fields, such as `HOURLYVISIBILITY`, that we may expect to be numerical, but are instead `object` type. This often indicates that there may be missing (or `null`) values, or some other unusual readings that we may have to deal with (since otherwise the field would have been fully parsed as a numerical data type).

In addition, some fields relate to hourly observations, while others relate to daily or monthly intervals. For purposes of later analysis, we will restrict the dataset to a certain subset  of fields that relate to hourly observations.

In this section, we refer to the [NOAA Local Climatological Data Documentation](https://data.noaa.gov/dataset/dataset/u-s-local-climatological-data-lcd/resource/ee7381ea-647a-434f-8cfa-81202b9b4c05) to describe the fields and meaning of various values.

#### Select data subset

First, we select only the subset of data of interest. We will keep data for years 2010 - 2017 related to routine hourly weather station reports. We will also restrict our dataset to only a subest of column types that we expect to be pertinent for downstream tasks.

In [ ]:
# Choose what columns to import from raw data
column_subset = [
    'DATE',
    'HOURLYVISIBILITY',
    'HOURLYPRSENTWEATHERTYPE',
    'HOURLYWindSpeed',
    'HOURLYWindGustSpeed',
    'HOURLYPrecip'
]

# Select the data sub-set for years 2010-2017 & report type FM-15 (routine hourly weather reports)
data_subset = raw_data[(raw_data['DATE'].dt.year.isin(range(2010, 2018))) & (raw_data['REPORTTPYE'] == 'FM-15')]
# Filter dataset to relevant columns
weather_data = data_subset.loc[:, column_subset]
# Set date index
weather_data = weather_data.set_index(pd.DatetimeIndex(weather_data['DATE']))
weather_data.drop(['DATE'], axis=1, inplace=True)
weather_data.replace(to_replace='*', value=np.nan, inplace=True)
weather_data.head()

In [ ]:
weather_data.dtypes

#### Clean up precipitation column

From the dataframe preview above, we can see that the column `HOURLYPrecip` - which is the hourly measure of precipitation levels - contains both `NaN` and `T` values. `T` specifies *trace amounts of precipitation*, while `NaN` means *not a number*, and is used to denote missing values.

We can also inspect the unique values present for the field.

In [ ]:
weather_data['HOURLYPrecip'].unique()

We can see that some values end with an `s` (indicating snow), while there is a strange value `0.020.01s` which appears to be an error of some sort. To deal with `T` values, we will set the observation to be `0`. We will also replace the erroneous value `0.020.01s` with `NaN`.

Finally, we will replace all `NaN` entries with `0`, i.e. we assume no precipitation was present.

In [ ]:
# Fix precipitation data
weather_data['HOURLYPrecip'].replace(to_replace='T', value='0.00', inplace=True)
weather_data['HOURLYPrecip'].replace('0.020.01s', np.nan, inplace=True)
weather_data.fillna(value={'HOURLYPrecip': 0}, inplace=True)

#### Inspect visibility column

As we have done for precipitation, we can also inspect the unique values present for the column `HOURLYVISIBILITY` - which is the hourly measure of visibility. Below, we see that some values are `nan`, while some end with an `V`. 

In [ ]:
weather_data['HOURLYVISIBILITY'].unique()

#### Convert columns to numerical types

Next, we will convert string columns that refer to numerical values to numerical types. For columns such as `HOURLYPrecip` and `HOURLYVISIBILITY`, we first also drop the non-numerical parts of the value (e.g .the `s` and `V` characters).

In [ ]:
# Set of columns to convert
messy_columns = ['HOURLYVISIBILITY', 'HOURLYPrecip', 'HOURLYWindSpeed', 'HOURLYWindGustSpeed']

# Convert columns to float32 datatype
for i in messy_columns:
    weather_data[i] = weather_data[i].apply(lambda x: re.sub('[^0-9,.-]', '', x) if type(x) == str else x).replace('', np.nan).astype(('float32'))

We can now see that all fields have numerical data type.

In [ ]:
print(weather_data.info())
# Generate the summary statistics for each column
weather_data.describe()

For wind gusts, rather than have `NaN` entries (which represent no gusts), we will represent the gust speed column as "excess speed" over the `HOURLYWindSpeed` values.weather_data.head()

In [ ]:
weather_data.loc[:, 'HOURLYWindGustSpeed'] = np.vectorize(lambda x, y: 0.0 if np.isnan(y) else y - x)(
    weather_data['HOURLYWindSpeed'], weather_data['HOURLYWindGustSpeed'])
weather_data.head()

#### Check date index

Next, we check if there are any duplicates with respect to our `DATE` index and check furthermore that our dates are in the correct order (that is, strictly increasing).

In [ ]:
cond = len(weather_data[weather_data.index.duplicated()].sort_index())
print('Date index contains no duplicate entries: {}'.format(cond == 0))
print('Date index is strictly increasing: {}'.format(weather_data.index.is_monotonic_increasing))

### Categorical Feature Extraction

The final pre-processing step we will perform will be to handle the `HOURLYPRSENTWEATHERTYPE` column to correctly encode the weather features. This column indicates the presence of specific weather types for the given reading. For example, `-RA:02 BR:1 |RA:61 |RA:61` refers to 3 types of reading:
1. `AU` codes for automated weather readings
2. `AW` codes for a different type of automated weather reading
3. `MW` codes for manually-augmented weather readings

This example reading happens to contain all 3 types, separated by a `|` character. The `AU` code is thus `-RA:02 BR:1`. If we refer to the data documentation linked above, we can see this indicates the presence of `RA:02 - Rain` and `BR:1 - Mist`.

These _present weather types_ are categorical variables. **Note** that multiple categories of weather can be present. In order to process this column, we will:
* only use the `AU` codes for simplicity
* convert the codes to more readable category names
* extract the weather type categories into individual binary columns representing the presence (`1`) or absence (`0`) of that category. This is like "one-hot encoding" but for multi-category variables

We start with creating a mapping from codes to category names

In [ ]:
# start with raw types taken from the LCD Dataset Documentation
# we convert the raw weather type names to snake_case
raw_types = '''DZ:01 - drizzle
RA:02 - rain
SN:03 - snow
SG:04 - snow_grains
IC:05 - ice_crystals
PL:06 - ice_pellets
GR:07 - hail
GS:08 - small_hail_snow_pellets
UP:09 - unknown_precipitation
BR:1 - mist
FG:2 - fog
FU:3 - smoke
VA:4 - volcanic_ash
DU:5 - widespread_dust
SA:6 - sand
HZ:7 - haze
PY:8 - spray
PO:1 - well_developed_dust
SQ:2 - squalls
FC:3 - funnel_cloud_waterspout_tornado
SS:4 - sandstorm
DS:5 - duststorm'''.split('\n')

raw_types = [t.split(' - ') for t in raw_types]
weather_types = {t[0]: t[1] for t in raw_types}
# Add in a code that is inconsistently represented in the documentation
weather_types['TS:7'] = 'thunderstorm'
weather_types

There are still a few edge cases that do not fall within the weather type mapping we have created. For the purposes of simplification, we will ignore these, since we have captured the main weather types in our mapping. So, we create a function to convert codes to category names, handling any errors.

In [ ]:
def get_type(k):
    if k in weather_types:
        return weather_types[k]
    else:
        return ''
    
def extract_weather_type(x):
    wt = x.split('|')[0].split() if isinstance(x, str) else []
    wt = [get_type(w.lstrip('-').lstrip('+')) for w in wt]
    return wt

Let's test our function out:

In [ ]:
print(weather_data['HOURLYPRSENTWEATHERTYPE'].head(5))
print()
print(weather_data['HOURLYPRSENTWEATHERTYPE'].apply(extract_weather_type).head(5))

That seems to be working. Next, we binarize the present weather categories in each cell:

In [ ]:
from collections import Counter
counts = weather_data['HOURLYPRSENTWEATHERTYPE'].apply(extract_weather_type).apply(Counter)
counts = pd.DataFrame.from_records(counts).fillna(value=0).drop(columns = [''])
counts

Finally, we combine the extra columns we've created with our original dataframe:

In [ ]:
cleaned_data = pd.concat([weather_data, counts.set_index(weather_data.index)], axis=1)
cleaned_data

#### Rename columns

Before saving the dataset, we will rename the columns for readability.

In [ ]:
cleaned_data.columns

In [ ]:
# define some new column names for consistency
columns_name_map = {
    'HOURLYVISIBILITY': 'visibility',
    'HOURLYPRSENTWEATHERTYPE': 'weather_type_raw',
    'HOURLYWindSpeed': 'wind_speed',
    'HOURLYWindGustSpeed': 'wind_gust_speed',
    'HOURLYPrecip': 'precip',
}

cleaned_data_renamed = cleaned_data.rename(columns=columns_name_map)

In [ ]:
print(cleaned_data_renamed.info())
print()
cleaned_data_renamed.head()

In [ ]:
# Log some general information about the dataset
print('# of columns: ' + str(cleaned_data_renamed.shape[1])) 
print('# of observations: ' + str(cleaned_data_renamed.shape[0]))
print('Start date: ' + str(cleaned_data_renamed.index[0]))
print('End date: ' + str(cleaned_data_renamed.index[-1]))

### Save the Processed Data

Finally, we save the processed dataset for use by downstream tasks.

In [ ]:
cleaned_data_renamed.to_csv('data/jfk_weather_features.csv', float_format='%g')

 ### Authors
 
 This notebook was created by the [Center for Open-Source Data & AI Technologies](http://codait.org).

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.